In [104]:
import requests

In [105]:
import geopandas as gpd
import pandas as pd

## Open County data

In [106]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
county_gdf = gpd.read_file(file_path)

## Get Columm Data

In [107]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/subject/variables.json")
columns_obj = r.json()

In [108]:
columns_to_replace = {
    key: val["label"]
    for key, val in columns_obj["variables"].items()
    if key.startswith("S0801")
}

In [109]:
mean_time_cols_dict = {
    col_name: col_dict
    for col_name, col_dict in columns_obj["variables"].items()
    if col_dict["label"].split("!!")[-1] == "Worked outside state of residence"
    and col_name.startswith("S0801")
}

In [110]:
mean_time_cols_dict

{'S0801_C02_017E': {'label': 'Estimate!!Male!!Workers 16 years and over!!PLACE OF WORK!!Worked outside state of residence',
  'concept': 'Commuting Characteristics by Sex',
  'predicateType': 'float',
  'group': 'S0801',
  'limit': 0,
  'attributes': 'S0801_C02_017EA,S0801_C02_017M,S0801_C02_017MA'},
 'S0801_C01_017E': {'label': 'Estimate!!Total!!Workers 16 years and over!!PLACE OF WORK!!Worked outside state of residence',
  'concept': 'Commuting Characteristics by Sex',
  'predicateType': 'float',
  'group': 'S0801',
  'limit': 0,
  'attributes': 'S0801_C01_017EA,S0801_C01_017M,S0801_C01_017MA'},
 'S0801_C03_017E': {'label': 'Estimate!!Female!!Workers 16 years and over!!PLACE OF WORK!!Worked outside state of residence',
  'concept': 'Commuting Characteristics by Sex',
  'predicateType': 'float',
  'group': 'S0801',
  'limit': 0,
  'attributes': 'S0801_C03_017EA,S0801_C03_017M,S0801_C03_017MA'}}

### Get columns to query and rename for later

In [111]:
columns = ["GEO_ID", "S0801_C01_017E", "S0801_C01_017M"]
columns_formatted = ",".join(columns)

In [112]:
response = requests.get(
    f"https://api.census.gov/data/2023/acs/acs5/subject?get={columns_formatted}&for=county:*"
)

In [113]:
response

<Response [200]>

In [114]:
data = response.json()
columns = data[0]
rows = data[1:]
commuting_df = pd.DataFrame(rows, columns=columns)

In [115]:
commuting_df = commuting_df[columns]
commuting_df[columns[1:]] = commuting_df[columns[1:]].astype(float)

In [116]:
def check_margin_error(row) -> str:
    val = row["S0801_C01_017E"]
    moe_val = row["S0801_C01_017M"]

    if val == 0.0:
        return val

    rmoe_val = abs(moe_val / val)
    if rmoe_val < 0.50:
        return float(val)
    else:
        return 0

In [117]:
commuting_df["percent_work_out_of_state"] = commuting_df.apply(
    lambda row: check_margin_error(row), axis=1
)

In [118]:
columns_to_replace["GEO_ID"] = "GEOIDFQ"
commuting_df = commuting_df.rename(columns=columns_to_replace)

## Merge Data

In [119]:
commuting_gdf = county_gdf.merge(commuting_df, on="GEOIDFQ", how="inner")

In [120]:
commuting_gdf = commuting_gdf.to_crs(9311)
commuting_gdf.to_file("data/Work_Out_Of_State_Per_County.gpkg")

In [121]:
commuting_gdf.sort_values("percent_work_out_of_state", ascending=False)

,STATEFP,COUNTYFP,COUNTYNS,GEOIDFQ,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,Estimate!!Total!!Workers 16 years and over!!PLACE OF WORK!!Worked outside state of residence,S0801_C01_017M,state,county,percent_work_out_of_state
2,01,113,00161583,0500000US01113,01113,Russell,Russell County,AL,Alabama,06,1660653961,15562947,"POLYGON ((1369018.524 -1290538.989, 1367863.98...",56.0,3.3,1.0,113.0,56.0
2123,37,073,01026126,0500000US37073,37073,Gates,Gates County,NC,North Carolina,06,882181709,13292008,"POLYGON ((2034804.217 -665136.674, 2034855.205...",52.1,5.4,37.0,73.0,52.1
2363,39,087,01074056,0500000US39087,39087,Lawrence,Lawrence County,OH,Ohio,06,1174282179,10029914,"POLYGON ((1483139.273 -560541.548, 1483674.464...",51.8,2.9,39.0,87.0,51.8
1307,27,027,00659459,0500000US27027,27027,Clay,Clay County,MN,Minnesota,06,2706993096,19303392,"POLYGON ((238969.738 243816.706, 240105.399 24...",51.4,2.0,27.0,27.0,51.4
773,05,091,00069166,0500000US05091,05091,Miller,Miller County,AR,Arkansas,06,1616257232,36848741,"POLYGON ((554219.49 -1251535.598, 554225.86 -1...",50.1,3.4,5.0,91.0,50.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,20,149,00485038,0500000US20149,20149,Pottawatomie,Pottawatomie County,KS,Kansas,06,2177493043,54149843,"POLYGON ((281326.334 -613918.628, 281559.404 -...",0.7,0.4,20.0,149.0,0.0
942,19,081,00465229,0500000US19081,19081,Hancock,Hancock County,IA,Iowa,06,1478895050,5265568,"POLYGON ((489978.776 -206609.333, 489879.304 -...",0.6,0.5,19.0,81.0,0.0
941,19,017,00465198,0500000US19017,19017,Bremer,Bremer County,IA,Iowa,06,1127886260,10165785,"POLYGON ((607478.171 -227762.734, 607231.143 -...",0.6,0.4,19.0,17.0,0.0
2028,19,095,00465236,0500000US19095,19095,Iowa,Iowa County,IA,Iowa,06,1518911686,2686847,"POLYGON ((638442.599 -337493.378, 638448.723 -...",0.1,0.1,19.0,95.0,0.0
